In [1]:
def parse_diskmap(data):
    diskmap = []
    file_id = 0
    for index, num in enumerate(data):
        num = int(num)
        if index % 2 == 0:
            diskmap.append((file_id, num))
            file_id += 1
        else:
            diskmap.append((None, num))
    return diskmap

def extend_diskmap(diskmap):
    extended = []
    for block_type, size in diskmap:
        extended.extend([block_type] * size)
    return extended

def initialize_disk_state(data):
    diskmap = parse_diskmap(data)
    disk_state = []
    blocks, spaces = [], []
    pos = 0

    for block_type, size in diskmap:
        if block_type is None:
            spaces.append((pos, size))
            disk_state.extend([None] * size)
        else:
            blocks.append((pos, size, block_type))
            disk_state.extend([block_type] * size)
        pos += size

    return diskmap, disk_state, blocks, spaces

def calculate_checksum(disk_state):
    return sum(i * val for i, val in enumerate(disk_state) if val is not None)


In [2]:
def calculate_checksum_part1(data):
    diskmap = parse_diskmap(data)
    extended = extend_diskmap(diskmap)
    
    # lower bound
    lb = 0
    for i in range(len(extended) - 1, 0, -1):
        if extended[i] is not None:
            for j in range(lb, i):
                if extended[j] is None:
                    lb = j
                    extended[j], extended[i] = extended[i], extended[j]
                    break

    return calculate_checksum(extended)

In [3]:
def calculate_checksum_part2(data):
    _, disk_state, blocks, spaces = initialize_disk_state(data)

    for pos, size, block_type in reversed(blocks):
        for i, (space_pos, space_size) in enumerate(spaces):
            if space_pos < pos and size <= space_size:
                for j in range(size):
                    assert disk_state[pos + j] == block_type
                    disk_state[pos + j] = None
                    disk_state[space_pos + j] = block_type


                if size == space_size:
                    # remove fully consumed space
                    spaces = [s for s in spaces if s != (space_pos, space_size)]
                else:
                    # Update remaining free space
                    spaces[i] = (space_pos + size, space_size - size)

                break

    return calculate_checksum(disk_state)

In [4]:
if __name__ == '__main__':
    with open("input.txt") as f:
        data = f.read().strip() 
    print("Part 1:", calculate_checksum_part1(data))
    print("Part 2:", calculate_checksum_part2(data))


Part 1: 6310675819476
Part 2: 6335972980679
